In [6]:
import numpy as np
import scipy as sp
import corner
from tqdm import tqdm
import os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

# Definitions

In [7]:
def correlation_matrix(m):
    """Calculates the correlation matrix of matrix m_ij."""
    avg = np.average(m, axis=0)
    dim = len(avg)
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # VARIATION
    sigma = []
    for i in range(dim):
        sigma.append(np.sum((avg[i] - m[i])**2))
    sigma = np.sqrt(sigma)
    
    sigma_bis = []
    for i in range(dim):
        cum = 0
        for j in range(len(m[i])):
            cum += (avg[i] - m[i][j])**2
        sigma_bis.append(cum)
    sigma_bis = np.sqrt(sigma_bis)        
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # CORRELATION MATRIX
    CORR = np.zeros((dim, dim))
    for i in range(dim):
        for j in range(dim):
            cum = 0
            for k in range(len(m[i])):
                cum += (avg[i] - m[i][k])*(avg[j] - m[j][k])
            CORR[i, j] = cum  / (sigma_bis[i] * sigma_bis[j])

    if np.linalg.det(CORR) == 0:
        print("¶ WARNING: correlation matrix is singular")
    
    return CORR

def Hartlap(mat, Nr = 350):
    """Calculates inverse matrix using Hartlap correction.
    Arguments:
    - `mat`: input matrix to invert
    - `Nr`: nuber of realization used o calculated the matrix
    """
    return (Nr-len(mat)-2)/(Nr-1)*np.linalg.inv(mat)

def info_name(name):
    """Obtain realization information from namefile"""
    print("TYPE: ", type(name))
    info = name.split('_')[-3:]
    print("TYPE: ", info, "\n", type(info))
    info[2] = info[2].replace(".wst", "")
    N_hgrid = info[0]
    N_WSTgrid = info[1]
    n_realiz = info[2].replace(".wst", "")
    
    return [int(N_hgrid), int(N_WSTgrid), int(n_realiz)]

def cosmo_parser(name):
    """Obtain cosmology from .wst file"""
    info = name.split('_')
    if info[0] == "fiducial":
        return info[0]
    elif info[0] == "zeldovich":
        return info[0]
    else:
        return info[0] + "_" + info[1]

In [8]:
order_folders = {
    'fiducial'  : 0,
    'h_m'       : 1,
    'h_p'       : 2,
    'Mnu_p'     : 3,
    'Mnu_pp'    : 4,
    'Mnu_ppp'   : 5,
    'ns_m'      : 6,
    'ns_p'      : 7,
    'Ob2_m'     : 8,
    'Ob2_p'     : 9,
    'Om_m'      : 10,
    'Om_p'      : 11,
    's8_m'      : 12,
    's8_p'      : 13,
    'w_m'       : 14,
    'w_p'       : 15,
    'zeldovich' : 16
}

cosmological_pars = {
    'Om'  : 0,
    'Ob'  : 1,
    'h'   : 2,
    'ns'  : 3,
    's8'  : 4,
    'w'   : 5,
    'Mnu' : 6
}

order_dimension = {
    'Om'  : 0,
    'Om ' : 0,
    'Ob'  : 1,
    'Ob ' : 1,
    'Ob2' : 1,
    'h'   : 2,
    'h  ' : 2,
    'ns'  : 3,
    'ns ' : 3,
    's8'  : 4,
    's8 ' : 4,
    'w'   : 5,
    'Mnu' : 6
}

COSMOPAR = {
#                   | Om   | Ob   |   h   |  n_s  | s_8 | Mnu | w |

    'fiducial' :    [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    'zeldovich':    [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    
    'Mnu_p' :       [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.1, -1],
    'Mnu_pp' :      [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.2, -1],
    'Mnu_ppp' :     [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0.4, -1],
    
    'h_m' :         [0.3175, 0.049, 0.6511, 0.9624, 0.834, 0, -1],
    'h_p' :         [0.3175, 0.049, 0.6911, 0.9624, 0.834, 0, -1],
    
    'ns_m' :        [0.3175, 0.049, 0.6711, 0.9424, 0.834, 0, -1],
    'ns_p' :        [0.3175, 0.049, 0.6711, 0.9824, 0.834, 0, -1],
    
    'Ob_m' :        [0.3175, 0.048, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob_p' :        [0.3175, 0.050, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob2_m' :       [0.3175, 0.047, 0.6711, 0.9624, 0.834, 0, -1],
    'Ob2_p' :       [0.3175, 0.051, 0.6711, 0.9624, 0.834, 0, -1],
    
    'Om_m' :        [0.3075, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    'Om_p' :        [0.3275, 0.049, 0.6711, 0.9624, 0.834, 0, -1],
    
    's8_m' :        [0.3175, 0.049, 0.6711, 0.9624, 0.819, 0, -1],
    's8_p' :        [0.3175, 0.049, 0.6711, 0.9624, 0.849, 0, -1],
    
    'w_m' :         [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -0.95],
    'w_p' :         [0.3175, 0.049, 0.6711, 0.9624, 0.834, 0, -1.05]
}
"""
Dictionary to assegnate to a cosmology its parameters
Ordered pairing cosmologies and the first one in with _m, the second one _p
"""

VarCosmoPar = {
    'd_h'  : 0.02,
    'd_ns' : 0.02,
    'd_Ob' : 0.001,
    'd_Ob2': 0.002,
    'd_Om' : 0.01,
    'd_s8' : 0.015,
    'd_w'  : -0.05
}
"""Values of deviations from fiducial cosmological parameters.
"""

fiducial_vals = {
    'Ob'  : 0.3175,
    'Ob2' : 0.3175,
    'Om'  : 0.049,
    'h'   : 0.6711,
    'n_s' : 0.9624,
    'ns'  : 0.9624,
    's_8' : 0.834,
    's8'  : 0.834,
    'Mnu' : 0,
    'w'   : -1
}
"""Values of fiducial cosmological parameters.
""";

In [13]:
root = "/media/fuffolo97/HDD1/UNI/Tesi/WST-files"
cosmologies = ['fiducial', 'Om_m', 'Om_p']
parameter_name = 'Om'
fiducial_coeffs_N = []

In [18]:
files_to_read_raw = os.listdir(root)
files_to_read_N_06_06 = []
files_to_read_N_06_08 = []
files_to_read_N_06_10 = []
files_to_read_N_08_06 = []
files_to_read_N_08_10 = []
files_to_read_N_10_06 = []
files_to_read_N_10_08 = []
files_to_read_N_10_10 = []
for i in range(len(files_to_read_raw)):
    if "s06" in files_to_read_raw[i] and "q06" in files_to_read_raw[i] and "_Ns_"  in files_to_read_raw[i]: files_to_read_N_06_06.append(files_to_read_raw[i])
    if "s06" in files_to_read_raw[i] and "q08" in files_to_read_raw[i] and "_Ns_"  in files_to_read_raw[i]: files_to_read_N_06_08.append(files_to_read_raw[i])
    if "s06" in files_to_read_raw[i] and "q10" in files_to_read_raw[i] and "_Ns_"  in files_to_read_raw[i]: files_to_read_N_06_10.append(files_to_read_raw[i])
    if "s08" in files_to_read_raw[i] and "q06" in files_to_read_raw[i] and "_Ns_"  in files_to_read_raw[i]: files_to_read_N_08_06.append(files_to_read_raw[i])
    if "s08" in files_to_read_raw[i] and "q10" in files_to_read_raw[i] and "_Ns_"  in files_to_read_raw[i]: files_to_read_N_08_10.append(files_to_read_raw[i])
    if "s10" in files_to_read_raw[i] and "q06" in files_to_read_raw[i] and "_Ns_"  in files_to_read_raw[i]: files_to_read_N_10_06.append(files_to_read_raw[i])
    if "s10" in files_to_read_raw[i] and "q08" in files_to_read_raw[i] and "_Ns_"  in files_to_read_raw[i]: files_to_read_N_10_08.append(files_to_read_raw[i])
    if "s10" in files_to_read_raw[i] and "q10" in files_to_read_raw[i] and "_Ns_"  in files_to_read_raw[i]: files_to_read_N_10_10.append(files_to_read_raw[i])

[]

In [ ]:
def coeff_calculator(files_list):
    files_to_read_N = files_list
    coeffs_tot_N = np.zeros((len(cosmologies), 75))
    zeldovich_coeffs_N = []
    fiducial_coeffs_N = []
    for i in range(len(files_to_read_N)):
        coeffs_cosm = []
        file_reading = files_to_read_N[i]

        cosmo = cosmo_parser(file_reading)
        assert cosmo in file_reading
        # print("COSMO KEY: ", cosmo)
        index = order_folders[cosmo]
        
        if ("fiducial" not in file_reading) and ("zeldovich" not in file_reading):
            # print("A, ", file_reading)
            with open(root + "/" + file_reading, 'rb') as Ff:
                while True:
                    try:
                        coeffs_cosm.append(pickle.load(Ff))
                    except EOFError:
                        break
                    except RuntimeWarning:
                        print("problem here: ", file_reading)

        elif "fiducial" in file_reading:
            # print("B, ", file_reading)
            with open(root + "/" + file_reading, 'rb') as Ff:
                while True:
                    try:
                        fiducial_coeffs_N.append(pickle.load(Ff))
                    except EOFError:
                        break
                    except RuntimeWarning:
                        print("problem here: ", file_reading)
            fiducial_coeffs_N_avg = np.average(fiducial_coeffs_N, axis=0)

            print(np.shape(fiducial_coeffs_N_avg))
            
            if len(fiducial_coeffs_N) == 0:
                print(cosmo, " has problem!")

        elif "zeldovich" in file_reading:
            # print("C, ", file_reading)
            with open(root + "/" + file_reading, 'rb') as Ff:
                while True:
                    try:
                        zeldovich_coeffs_N.append(pickle.load(Ff))
                    except EOFError:
                        break
                    except RuntimeWarning:
                        print("problem here: ", file_reading)
            # print(zeldovich_coeffs)
            zeldovich_coeffs_N_avg = np.average(zeldovich_coeffs_N, axis=0)
            # print(zeldovich_coeffs_avg)
            print(np.shape(zeldovich_coeffs_N_avg))
        else:
            assert False, "\n   ERROR in reading WST files, cosmology can't be propely interpreted.\n"

        # USING average  value per coefficient in cosmology
        if len(coeffs_cosm) == 0 and cosmo != "fiducial" and cosmo != "zeldovich":
            print(cosmo, " has problem!")
            print(cosmo, " : ", np.shape(coeffs_cosm))
        if cosmo != "fiducial" and cosmo != "zeldovich":
            coeffs_tot_N[index] = np.average(coeffs_cosm, axis=0)
    
    return (coeffs_tot_N, fiducial_coeffs_N_avg, fiducial_coeffs_N, zeldovich_coeffs_N_avg, zeldovich_coeffs_N)